In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Rg4nvBAHF5rS2Z6bvfDd5SNp-O9G9hhfOSrQ2ksYLCc&tc=ExTkswg_gu87j8HVyP_ztiInKBRbr7AAWetMuD1gXgs&cc=TJUwFrmOu5eSoi31EDxDOu3l9Eexi5n9H9hs28dPvpY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkmWBpa94R9okwxyERRkr0Ae4ikQWx-PWL450fZCuu1SLzQMhQ0bKo

Successfully saved authorization token.


In [17]:
# MODIS Burned Area
# The Terra and Aqua combined MCD64A1 Version 6 Burned Area data product is a monthly,
# global gridded 500 meter (m) product containing per-pixel burned-area

dataset = ee.ImageCollection('MODIS/006/MCD64A1')\
.filter(ee.Filter.date('2001-01-01', '2022-12-31'))\
.median()

burnedArea = dataset.select('BurnDate')

burnedAreaVis = {'min': 30.0, 'max': 341.0, 'palette': ['4e0400', '951003', 'c61503', 'ff1901']}

In [18]:
#Load country data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Australia = countries.filter(ee.Filter.eq('country_na', 'Australia'))

In [19]:
import folium

In [20]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [21]:
# Create a folium map object.
my_map = folium.Map(location=[-25.2744, 133.7751], zoom_start = 4)

# Add the layer to the map object.
my_map.add_ee_layer(burnedArea.clip(Australia), burnedAreaVis, 'Burned Area 2015-2022')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)